# Gliederung Prompt Engineering
### Starte mit einem generischen Prompt
### Füge eine professionelle Rolle hinzu
### Führe konkrete Einschränkungen ein
### Bonus: Wende an, was du gelernt hast

In [3]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables and initialize OpenAI client
load_dotenv()

# Azure OpenAI Configuration - following the pattern you showed
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# Model name should match your Azure deployment name
model = azure_deployment if azure_deployment else "gpt-4o-mini"

if not api_key or not azure_endpoint:
    raise ValueError("Azure OpenAI configuration missing. Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, and AZURE_OPENAI_DEPLOYMENT_NAME in your .env file.")

# Configure client for Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model=model):
    """Simple wrapper for OpenAI API calls."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [4]:
# No changes needed in this cell
from enum import Enum


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = "gpt-4.1-mini"

# COMPLETION FUNCTION
def get_completion(system_prompt, user_prompt, model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt
        user_prompt: The user prompt
        model: The model to use (default is gpt-4.1-mini)
    Returns:
        The completion text
    """
    messages = [
        {"role": "user", "content": user_prompt},
    ]
    if system_prompt is not None:
        messages = [
            {"role": "system", "content": system_prompt},
            *messages,
        ]
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

# RESPONSE DISPLAY FUNCTION
def display_responses(*args):
    """Helper function to display responses as Markdown, horizontally."""
    markdown_string = "<table><tr>"
    # Headers
    for arg in args:
        markdown_string += f"<th>System Prompt:<br />{arg['system_prompt']}<br /><br />"
        markdown_string += f"User Prompt:<br />{arg['user_prompt']}</th>"
    markdown_string += "</tr>"
    # Rows
    markdown_string += "<tr>"
    for arg in args:
        markdown_string += f"<td>Response:<br />{arg['response']}</td>"
    markdown_string += "</tr></table>"
    display(Markdown(markdown_string))

## 1. Generic Prompt

First, let's see what the model produces with a basic prompt.

In [5]:
# No changes needed in this cell
plain_system_prompt = "Du hilft Probleme zu lösen."  # A generic system prompt
user_prompt = "Mein Computer startet nicht mehr. Was kann ich tun?"

print(f"Sending prompt to {MODEL} model...")
baseline_response = get_completion(plain_system_prompt, user_prompt)
print("Response received!\n")

display_responses(
    {
        "system_prompt": plain_system_prompt,
        "user_prompt": user_prompt,
        "response": baseline_response,
    }
)

Sending prompt to gpt-4.1-mini model...
Response received!



<table><tr><th>System Prompt:<br />Du hilft Probleme zu lösen.<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th></tr><tr><td>Response:<br />Wenn dein Computer nicht mehr startet, kannst du folgende Schritte ausprobieren, um das Problem zu beheben:

1. **Stromversorgung prüfen**  
   - Stelle sicher, dass das Netzkabel richtig eingesteckt ist.  
   - Überprüfe, ob die Steckdose funktioniert, indem du ein anderes Gerät anschließt.  
   - Bei einem Laptop: Prüfe, ob der Akku geladen ist und das Netzteil funktioniert.

2. **Hardware-Verbindungen kontrollieren**  
   - Öffne das Gehäuse (bei Desktop-PCs) und kontrolliere, ob alle Kabel und Komponenten (RAM, Grafikkarte, Festplatte) fest sitzen.  
   - Bei Laptops nur, wenn du dich damit auskennst und es nicht die Garantie verletzt.

3. **Peripheriegeräte entfernen**  
   - Trenne alle externen Geräte (USB-Sticks, Drucker, externe Festplatten) und versuche erneut zu starten.

4. **Fehlermeldungen oder Signaltöne beachten**  
   - Achte beim Einschalten auf Pieptöne oder Fehlercodes, die auf ein bestimmtes Problem hinweisen können.

5. **Bildschirm prüfen**  
   - Vergewissere dich, dass der Monitor eingeschaltet ist und richtig angeschlossen ist.

6. **Versuche den abgesicherten Modus**  
   - Falls das System zumindest startet, versuche mit der Taste F8 (bei Windows) oder einer passenden Tastenkombination in den abgesicherten Modus zu starten.

7. **CMOS Batterie resetten**  
   - Entferne kurzzeitig die CMOS-Batterie (bei Desktop-PCs) um BIOS-Einstellungen zurückzusetzen.

8. **Externe Hilfe**  
   - Wenn keiner der Schritte hilft, kann es ein Hardwaredefekt sein (Netzteil, Mainboard, Festplatte). Dann ist es ratsam, einen Fachmann zu konsultieren.

Wenn du mir mehr Details gibst (z.B. was genau passiert beim Einschalten, ob LEDs leuchten oder Geräusche zu hören sind), kann ich gezielter helfen.</td></tr></table>

## 2. Add a Professional Role

Now, let's add a professional role to see how it affects the response.

<div style="color: red">NOTE: We will use the same user prompt for these examples. All you need to do is vary the system prompt, which is where one would normally define the role for an LLM.</div>

In [ ]:
role_system_prompt = """
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
"""


print("Sending prompt with professional role...")
role_response = get_completion(role_system_prompt, user_prompt)
print("Response received!\n")

# Show last two prompts and responses
display_responses(
    {
        "system_prompt": plain_system_prompt,
        "user_prompt": user_prompt,
        "response": baseline_response,
    },
    {
        "system_prompt": role_system_prompt,
        "user_prompt": user_prompt,
        "response": role_response,
    },
)

Sending prompt with professional role...
Response received!



<table><tr><th>System Prompt:<br />Du hilft Probleme zu lösen.<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th><th>System Prompt:<br />
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th></tr><tr><td>Response:<br />Wenn dein Computer nicht mehr startet, kannst du folgende Schritte ausprobieren, um das Problem zu beheben:

1. **Stromversorgung prüfen**  
   - Stelle sicher, dass das Netzkabel richtig eingesteckt ist.  
   - Überprüfe, ob die Steckdose funktioniert, indem du ein anderes Gerät anschließt.  
   - Bei einem Laptop: Prüfe, ob der Akku geladen ist und das Netzteil funktioniert.

2. **Hardware-Verbindungen kontrollieren**  
   - Öffne das Gehäuse (bei Desktop-PCs) und kontrolliere, ob alle Kabel und Komponenten (RAM, Grafikkarte, Festplatte) fest sitzen.  
   - Bei Laptops nur, wenn du dich damit auskennst und es nicht die Garantie verletzt.

3. **Peripheriegeräte entfernen**  
   - Trenne alle externen Geräte (USB-Sticks, Drucker, externe Festplatten) und versuche erneut zu starten.

4. **Fehlermeldungen oder Signaltöne beachten**  
   - Achte beim Einschalten auf Pieptöne oder Fehlercodes, die auf ein bestimmtes Problem hinweisen können.

5. **Bildschirm prüfen**  
   - Vergewissere dich, dass der Monitor eingeschaltet ist und richtig angeschlossen ist.

6. **Versuche den abgesicherten Modus**  
   - Falls das System zumindest startet, versuche mit der Taste F8 (bei Windows) oder einer passenden Tastenkombination in den abgesicherten Modus zu starten.

7. **CMOS Batterie resetten**  
   - Entferne kurzzeitig die CMOS-Batterie (bei Desktop-PCs) um BIOS-Einstellungen zurückzusetzen.

8. **Externe Hilfe**  
   - Wenn keiner der Schritte hilft, kann es ein Hardwaredefekt sein (Netzteil, Mainboard, Festplatte). Dann ist es ratsam, einen Fachmann zu konsultieren.

Wenn du mir mehr Details gibst (z.B. was genau passiert beim Einschalten, ob LEDs leuchten oder Geräusche zu hören sind), kann ich gezielter helfen.</td><td>Response:<br />Danke für die Information. Um die Ursache des Problems eingrenzen zu können, brauche ich noch ein paar Details:

1. Welche Art von Computer ist es (Laptop, Desktop, Marke/Modell)?
2. Gibt es beim Starten irgendwelche Fehlermeldungen oder piept der PC?
3. Passiert überhaupt nichts, oder startet der Rechner, aber der Bildschirm bleibt schwarz?
4. Wurde kürzlich Hardware geändert oder Software installiert?
5. Funktioniert die Stromversorgung (z.B. Netzteil, Akkustand bei Laptop)?

Basierend auf den Antworten kann ich eine gezieltere Diagnose und Lösung vorschlagen. Generell empfehle ich als erste Schritte:

- Überprüfen der Stromversorgung (Netzkabel, Netzteil, Akku)
- Entfernen externer Geräte (USB, Peripherie)
- Falls möglich, CMOS-Reset durchführen (Batterie kurz entfernen oder Jumper setzen)
- Prüfen, ob die Hardware (RAM, Festplatte) korrekt sitzt

Gib mir gerne mehr Details, dann helfe ich dir weiter!</td></tr></table>

## 3. Ausgabeformat zur Weiterverarbeitung

Damit wir den Output des Agents gut parsen und weiterverarbeiten können, bitten wir ihn den Output im JSON FORMAT auszugeben.

In [7]:
# 3. Ausgabeformat zur Weiterverarbeitung
formated_system_prompt = f""" {role_system_prompt}. 
# Output Format
Bitte gebe deine Antwort im folgenden JSON Format. Gebe nur das JSON Objekt zurück, ohne weitere Erklärungen oder Text.
FORMAT:
{{
  "Ursache": "<mögliche Ursache des Problems>",
  "Empfehlung": "<konkrete Handlungsempfehlung zur Problemlösung>"
}}
EXAMPLE:
Hier ist ein Beispiel:
Input: "Mein Computer startet nicht mehr. Was kann ich tun?"
Output:
{{
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers oder eines Softwareproblems nicht starten.",
  "Empfehlung": "Überprüfen Sie die Stromversorgung und alle Kabelverbindungen. Versuchen Sie, den Computer im abgesicherten Modus zu starten. Wenn das Problem weiterhin besteht, führen Sie eine Systemwiederherstellung durch oder kontaktieren Sie den technischen Support."
}}
"""

print("Sending prompt with constraints...")
formated_response = get_completion(formated_system_prompt, user_prompt)
print("Response received!\n")

# Show last two prompts and responses
display_responses(
    {
        "system_prompt": role_system_prompt,
        "user_prompt": user_prompt,
        "response": role_response,
    },
    {
        "system_prompt": formated_system_prompt,
        "user_prompt": user_prompt,
        "response": formated_response,
    },
)

Sending prompt with constraints...
Response received!



<table><tr><th>System Prompt:<br />
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th><th>System Prompt:<br /> 
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
. 
# Output Format
Bitte gebe deine Antwort im folgenden JSON Format. Gebe nur das JSON Objekt zurück, ohne weitere Erklärungen oder Text.
FORMAT:
{
  "Ursache": "<mögliche Ursache des Problems>",
  "Empfehlung": "<konkrete Handlungsempfehlung zur Problemlösung>"
}
EXAMPLE:
Hier ist ein Beispiel:
Input: "Mein Computer startet nicht mehr. Was kann ich tun?"
Output:
{
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers oder eines Softwareproblems nicht starten.",
  "Empfehlung": "Überprüfen Sie die Stromversorgung und alle Kabelverbindungen. Versuchen Sie, den Computer im abgesicherten Modus zu starten. Wenn das Problem weiterhin besteht, führen Sie eine Systemwiederherstellung durch oder kontaktieren Sie den technischen Support."
}
<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th></tr><tr><td>Response:<br />Danke für die Information. Um die Ursache des Problems eingrenzen zu können, brauche ich noch ein paar Details:

1. Welche Art von Computer ist es (Laptop, Desktop, Marke/Modell)?
2. Gibt es beim Starten irgendwelche Fehlermeldungen oder piept der PC?
3. Passiert überhaupt nichts, oder startet der Rechner, aber der Bildschirm bleibt schwarz?
4. Wurde kürzlich Hardware geändert oder Software installiert?
5. Funktioniert die Stromversorgung (z.B. Netzteil, Akkustand bei Laptop)?

Basierend auf den Antworten kann ich eine gezieltere Diagnose und Lösung vorschlagen. Generell empfehle ich als erste Schritte:

- Überprüfen der Stromversorgung (Netzkabel, Netzteil, Akku)
- Entfernen externer Geräte (USB, Peripherie)
- Falls möglich, CMOS-Reset durchführen (Batterie kurz entfernen oder Jumper setzen)
- Prüfen, ob die Hardware (RAM, Festplatte) korrekt sitzt

Gib mir gerne mehr Details, dann helfe ich dir weiter!</td><td>Response:<br />{
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers (z.B. Netzteil, Festplatte, RAM) oder eines Softwareproblems (z.B. beschädigtes Betriebssystem) nicht starten.",
  "Empfehlung": "Überprüfen Sie zunächst die Stromversorgung und alle Kabelverbindungen. Testen Sie das Netzteil und tauschen Sie es ggf. aus. Starten Sie den Computer im abgesicherten Modus oder von einem Boot-Medium, um die Integrität des Betriebssystems zu prüfen. Führen Sie bei Bedarf eine Systemreparatur oder Neuinstallation durch."
}</td></tr></table>

## LLM as a Judge

Wir nutzen nun ein weiteres LLM um den Output zu überprüfen.

In [8]:
judge_system_prompt = """
  Du bist ein sorgfältiger und detailorientierter Evaluator.
  Deine Aufgabe ist es, die Antworten eines anderen KI-Modells
   anhand folgender Kriterien zu bewerten:

  1. **Korrektheit**: Sind die bereitgestellten Informationen 
  richtig und relevant für das Problem des Benutzers?
  2. **Klarheit**: Ist die Antwort klar und leicht 
  verständlich?
  3. **Nützlichkeit**: Bietet die Antwort umsetzbare
  Ratschläge, die der Benutzer zur Lösung seines Problems
  befolgen kann?
  4. **Format-Konformität**: Entspricht die Antwort dem
  angeforderten JSON-Format ohne zusätzlichen Text oder
  Erklärungen?

  Bewerte jeden Aspekt mit einer Punktzahl von 1 bis 10 und       
  gib eine kurze Begründung.
  Strukturiere deine Bewertung übersichtlich und abschließend     
  mit einer Gesamtbewertung.

  Format deiner Antwort:
  - Verwende klare Überschriften
  - Strukturiere mit Aufzählungen
  - Gib am Ende eine Gesamtbewertung und Zusammenfassung
  """

  # Verwende die formatierte Antwort aus der vorherigen Zelle zur Bewertung
judge_user_prompt = f"""
  Bitte bewerte die folgende Antwort:

  {formated_response}

  Die ursprüngliche Benutzeranfrage war: "{user_prompt}"
  """
print("Sende an LLM-Richter zur Bewertung...")
judge_response = get_completion(judge_system_prompt,
judge_user_prompt)
print("Bewertung erhalten!\n")

# Zeige mit display_responses Funktion
display_responses(
      {
          "system_prompt": judge_system_prompt,
          "user_prompt": judge_user_prompt,
          "response": judge_response,
      }
  )


Sende an LLM-Richter zur Bewertung...
Bewertung erhalten!



<table><tr><th>System Prompt:<br />
  Du bist ein sorgfältiger und detailorientierter Evaluator.
  Deine Aufgabe ist es, die Antworten eines anderen KI-Modells
   anhand folgender Kriterien zu bewerten:

  1. **Korrektheit**: Sind die bereitgestellten Informationen 
  richtig und relevant für das Problem des Benutzers?
  2. **Klarheit**: Ist die Antwort klar und leicht 
  verständlich?
  3. **Nützlichkeit**: Bietet die Antwort umsetzbare
  Ratschläge, die der Benutzer zur Lösung seines Problems
  befolgen kann?
  4. **Format-Konformität**: Entspricht die Antwort dem
  angeforderten JSON-Format ohne zusätzlichen Text oder
  Erklärungen?

  Bewerte jeden Aspekt mit einer Punktzahl von 1 bis 10 und       
  gib eine kurze Begründung.
  Strukturiere deine Bewertung übersichtlich und abschließend     
  mit einer Gesamtbewertung.

  Format deiner Antwort:
  - Verwende klare Überschriften
  - Strukturiere mit Aufzählungen
  - Gib am Ende eine Gesamtbewertung und Zusammenfassung
  <br /><br />User Prompt:<br />
  Bitte bewerte die folgende Antwort:

  {
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers (z.B. Netzteil, Festplatte, RAM) oder eines Softwareproblems (z.B. beschädigtes Betriebssystem) nicht starten.",
  "Empfehlung": "Überprüfen Sie zunächst die Stromversorgung und alle Kabelverbindungen. Testen Sie das Netzteil und tauschen Sie es ggf. aus. Starten Sie den Computer im abgesicherten Modus oder von einem Boot-Medium, um die Integrität des Betriebssystems zu prüfen. Führen Sie bei Bedarf eine Systemreparatur oder Neuinstallation durch."
}

  Die ursprüngliche Benutzeranfrage war: "Mein Computer startet nicht mehr. Was kann ich tun?"
  </th></tr><tr><td>Response:<br />### Bewertung der Antwort

**1. Korrektheit (9/10)**  
- Die Antwort nennt relevante und korrekte Ursachen für das Problem (Hardware- und Softwarefehler).  
- Die empfohlenen Schritte sind technisch sinnvoll und entsprechen bewährten Vorgehensweisen zur Fehlerbehebung bei einem nicht startenden Computer.  
- Ein kleiner Punktabzug, da die Antwort keine weiteren Fehlerquellen (z.B. BIOS/UEFI-Probleme) erwähnt, die ebenfalls relevant sein könnten.

**2. Klarheit (9/10)**  
- Die Antwort ist klar formuliert und gut verständlich, auch für technisch weniger versierte Nutzer.  
- Die Trennung in Ursache und Empfehlung ist sinnvoll und strukturiert.  
- Die Empfehlungen könnten noch etwas einfacher erklärt werden (z.B. was genau „abgesicherter Modus“ bedeutet).

**3. Nützlichkeit (9/10)**  
- Die Antwort bietet konkrete, umsetzbare Schritte zur Problemlösung, die der Benutzer ausprobieren kann.  
- Die Hinweise sind praxisnah und decken mehrere mögliche Ursachen ab.  
- Ein zusätzlicher Hinweis auf den Support oder Fachpersonal hätte die Nützlichkeit noch erhöht.

**4. Format-Konformität (10/10)**  
- Die Antwort entspricht exakt dem angeforderten JSON-Format.  
- Es gibt keine zusätzlichen Erklärungen oder Text außerhalb des JSON-Blocks.

---

### Gesamtbewertung: 9.25/10

**Zusammenfassung:**  
Die Antwort ist inhaltlich korrekt, klar verständlich und liefert nützliche, umsetzbare Ratschläge zur Problembehebung. Das JSON-Format wurde perfekt eingehalten. Kleine Verbesserungen wären eine ausführlichere Erklärung technischer Begriffe und eine erweiterte Ursachenanalyse. Insgesamt ist die Antwort sehr gut geeignet, um einem Nutzer bei einem nicht startenden Computer weiterzuhelfen.</td></tr></table>